In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import scipy
from scipy import signal
import plotly.graph_objects as go


In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']



default = ['South Africa', 'United Kingdom', 'Germany', 'Singapore', 'Australia', 'Italy', 
        'Japan', 'Sweden', 'New Zealand', "Afghanistan"]

In [3]:
def data_prep():
    
    df = pd.read_csv('time_series_covid19_confirmed_global.csv')
    
    countries = df['Country/Region'].unique()
    
    df.drop(['Province/State', 'Lat', 'Long'], inplace=True, axis=1)
    df = df.melt(id_vars=['Country/Region'], value_name='Confirmed', var_name='Date')
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')
    df = df.groupby(['Country/Region', 'Date']).sum().reset_index()

    df = df[df['Confirmed'] >= 1]
    df
    df.sort_values(by=['Country/Region', 'Date'], inplace=True)

    newDf = pd.DataFrame()
    for c in countries:
        tmp = df[df['Country/Region'] == c]
        dateList = []
        for i in np.arange(0, tmp.shape[0], 1):
            dateList.append((tmp.iloc[i, 1] - tmp.iloc[0, 1]).days)
        tmp['DaysFromFirstConf'] = dateList
        tmp['DailyNewConf'] = tmp['Confirmed'].diff()

        newDf = newDf.append(tmp)  
    #placesDict = [{'label': k, 'value': k} for k in countries]   
    return newDf, countries

In [4]:
# prep the data
newDf, countries, = data_prep()

# create soem colours
colors = px.colors.qualitative.Bold

# Build App
app = JupyterDash(__name__)

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

#####################################################################################
## Layout
app.layout = html.Div(children=[   
    
    html.H1(children='COVID-19 rates in countries',
           style={'textAlign': 'center'}),  
    
    html.Label('Countrie(s)'),
    
    dcc.Dropdown(
        id='selection',
        options=[{'label': v, 'value': v} for v in countries],
        value=default,
        multi=True
    ),
    
    # Output figures
    ## Cumulative           
    dcc.Graph(id='cumulative'),    
    ## Daily New
    dcc.Graph(id='Daily_new'),
    ## Delivative
    dcc.Graph(id='d1'),
    
    html.Br()
])

# Cumulative figure
#####################################################################################
@app.callback(
    Output('cumulative', 'figure'),    
    [Input('selection', 'value')])

def update_cum(default):
    keep = default
    fig_cum = go.Figure()
    i = 0
    for c in keep:    
        tmp = newDf[newDf['Country/Region'] == c]
        fig_cum.add_trace(go.Scatter(
            x=tmp['DaysFromFirstConf'],
            y=tmp['Confirmed'],
            mode='lines',
            marker=dict(size=5, color=colors[i]),
            hovertext=tmp['Date'].dt.date,
            name=c
        ))
        i += 1

    fig_cum.update_layout(yaxis_type="log",
                      template="plotly_white", 
                      title="Cumulative COVID-19 cases",
                      yaxis_title="Cumulative cases",
                      xaxis_title="Days from first",
                      width=1050,
                      height=600)
    return fig_cum


# Daily new
#####################################################################################
@app.callback(
    Output('Daily_new', 'figure'),    
    [Input('selection', 'value')])

def update_new(default):
    keep = default
    fig_new = go.Figure()
    i = 0
    for c in keep:
        tmp = newDf[newDf['Country/Region'] == c]
        fig_new.add_trace(go.Scatter(
            x=tmp['DaysFromFirstConf'],
            y=tmp['DailyNewConf'],
            mode='lines+markers',
            marker=dict(size=5, color=colors[i]),
            line = dict(dash='dot'),
            hovertext=tmp['Date'].dt.date,
            name=c
        ))

        fig_new.add_trace(go.Scatter(
            x=tmp['DaysFromFirstConf'],
            y=signal.savgol_filter(tmp['DailyNewConf'],
                                  window_length = 21,
                                  polyorder=2, deriv=0),
            mode='lines',
            opacity=0.7,
            marker=dict(size=5, color=colors[i]),
            hovertext=tmp['Date'].dt.date,
            name=f'{c} smoothed'    
        ))   
        i += 1

    fig_new.update_layout(template="plotly_white", 
                      title="New COVID-19 cases",
                      yaxis_title="Daily new",
                      xaxis_title="Days from first",
                      width=1050,
                      height=600)
    return fig_new


#Delta change
#####################################################################################
@app.callback(
    Output('d1', 'figure'),    
    [Input('selection', 'value')])

def update_d1(default):
    keep = default
    fig_d1 = go.Figure()
    i = 0
    for c in keep:    
        tmp = newDf[newDf['Country/Region'] == c]    
        fig_d1.add_trace(go.Scatter(
            x=tmp['DaysFromFirstConf'],
            y=signal.savgol_filter(tmp['DailyNewConf'],
                                  window_length = 21,
                                  polyorder=2, deriv=1),
            mode='lines+markers',
            opacity=0.7,
            marker=dict(size=5, color=colors[i]),
            hovertext=tmp['Date'].dt.date,
            name=f'{c} delta cases'            
        ))   
        i += 1

    fig_d1.update_layout(template="plotly_white", 
                      title="First derivative of new COVID-19 cases",
                      yaxis_title="Change in new cases",
                      xaxis_title="Days from first",
                      width=1050,
                      height=600)

    return fig_d1

app.run_server(mode='external')

/home/armand/.local/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/armand/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Dash app running on http://127.0.0.1:8050/
